# RSI momentum trading strategy example

- This is a backtest example notebook
    - New style grid search

# Set up

Set up Trading Strategy data client.


In [ ]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client

client = Client.create_jupyter_client()


# Render for Github web viewer
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)
# etup_charting_and_output(width=1500, height=1000)

# Load data

We use Binance data so we get a longer period of data.

In [ ]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.chain import ChainId
from tradeexecutor.utils.binance import create_binance_universe

target_time_bucket = TimeBucket.h8  # Upsample shifted data to this time bucket

strategy_universe = create_binance_universe(
    ["BTCUSDT", "ETHUSDT"],   # Binance internal tickers later mapped to Trading strategy DEXPair metadata class
    candle_time_bucket=TimeBucket.h8,
    stop_loss_time_bucket=TimeBucket.h1,
    start_at=datetime.datetime(2019, 1, 1),  # Backtest for 5 years data
    end_at=datetime.datetime(2024, 2, 15),
    include_lending=False
)

# List of pair descriptions we used to look up pair metadata
our_pairs = [
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
]



# Show loaded trading universe

Display generic troubleshooting information about the loaded data.

In [ ]:
from tradingstrategy.chain import ChainId

pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"Loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    pair_candles = candles.get_candles_by_pair(pair)
    first_close = pair_candles.iloc[0]["close"]
    first_close_at = pair_candles.index[0]
    print(f"Pair {pair} first close price {first_close} at {first_close_at}")


# Indicators

- Prepare indicators needed for the grid

In [ ]:
import pandas_ta

from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse


def calculate_eth_btc(strategy_universe: TradingStrategyUniverse):
    eth = strategy_universe.get_pair_by_human_description(our_pairs[0])
    btc = strategy_universe.get_pair_by_human_description(our_pairs[1])    
    btc_price = strategy_universe.data_universe.candles.get_candles_by_pair(eth.internal_id)
    eth_price = strategy_universe.data_universe.candles.get_candles_by_pair(btc.internal_id)
    series = eth_price["close"] / btc_price["close"]  # Divide two series
    return series

def calculate_eth_btc_rsi(strategy_universe: TradingStrategyUniverse, length: int):
    weth_usdc = strategy_universe.get_pair_by_human_description((ChainId.ethereum, "test-dex", "WETH", "USDC"))
    wbtc_usdc = strategy_universe.get_pair_by_human_description((ChainId.ethereum, "test-dex", "WBTC", "USDC"))
    btc_price = strategy_universe.data_universe.candles.get_candles_by_pair(wbtc_usdc.internal_id)
    eth_price = strategy_universe.data_universe.candles.get_candles_by_pair(weth_usdc.internal_id)
    eth_btc = eth_price["close"] / btc_price["close"]
    return pandas_ta.rsi(eth_btc, length=length)

def create_indicators(parameters: StrategyParameters, indicators: IndicatorSet, strategy_universe: TradingStrategyUniverse, execution_context: ExecutionContext):
    # Calculate RSI for all pairs
    indicators.add("rsi", pandas_ta.rsi, {"length": parameters.rsi_length})  
    # Custom: ETC/BTC close price
    indicators.add("eth_btc", calculate_eth_btc, source=IndicatorSource.strategy_universe)
    # Custom: ETC/BTC RSI
    # indicators.add("eth_btc_rsi", calculate_eth_btc_rsi, parameters={"length": parameters.eth_btc_rsi_length}, source=IndicatorSource.strategy_universe)

# Trading algorithm

In [ ]:
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput
from tradingstrategy.utils.groupeduniverse import resample_candles
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.weighting import weight_equal, weight_by_1_slash_n, weight_passthrouh
from tradeexecutor.strategy.alpha_model import AlphaModel
import numpy as np
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradingstrategy.chain import ChainId
from typing import List, Dict

import pandas as pd

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.state.visualisation import PlotKind, PlotShape, PlotLabel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.state.state import State



def decide_trades(
    input: StrategyInput,
) -> List[TradeExecution]:
    
    # Resolve our pair metadata for our two pair strategy
    parameters = input.parameters 
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators = input.indicators
    
    alpha_model = AlphaModel(input.timestamp)
    btc_pair = position_manager.get_trading_pair(our_pairs[0])
    eth_pair = position_manager.get_trading_pair(our_pairs[1])
    position_manager.log("decide_trades() start")

    #
    # Indicators
    #
    # Calculate indicators for each pair.
    #

    # Per-trading pair calcualted data
    current_rsi_values = {}  # RSI yesterday
    previous_rsi_values = {}  # RSI day before yesterday
    current_price = {}  # Close price yesterday
    momentum = {btc_pair: 0, eth_pair: 0}
    eth_btc_rsi_yesterday = None

    for pair in [btc_pair, eth_pair]:        
        current_price[pair] = indicators.get_price(pair)
        current_rsi_values[pair] = indicators.get_indicator_value("rsi", pair=pair)
        previous_rsi_values[pair] = indicators.get_indicator_value("rsi", index=-2, pair=pair)

    eth_btc_yesterday = indicators.get_indicator_value("eth_btc")
    eth_btc_rsi_yesterday = indicators.get_indicator_value("eth_btc_rsi")
    if eth_btc_rsi_yesterday is not None:
        momentum[eth_pair] = (eth_btc_rsi_yesterday - 50) ** 3
        momentum[btc_pair] = (50 - momentum[eth_pair]) ** 3

    #
    # Trading logic
    #

    for pair in [btc_pair, eth_pair]:
        existing_position = position_manager.get_current_position_for_pair(pair)
        pair_open = existing_position is not None
        pair_momentum = momentum.get(pair, 0)
        signal_strength = max(pair_momentum, 0.1)  # Singal strength must be positive, as we do long-only
        if pd.isna(signal_strength):
            signal_strength = 0
        alpha_model.set_signal(pair, 0)

        if pair_open:
            # We have existing open position for this pair,
            # keep it open by default unless we get a trigger condition below
            position_manager.log(f"Pair {pair} already open")
            alpha_model.set_signal(pair, signal_strength, trailing_stop_loss=parameters.trailing_stop_loss)

        if current_rsi_values[pair] and previous_rsi_values[pair]:

            # Check for RSI crossing our threshold values in this cycle, compared to the previous cycle
            rsi_cross_above = current_rsi_values[pair] >= parameters.rsi_high and previous_rsi_values[pair] < parameters.rsi_high
            rsi_cross_below = current_rsi_values[pair] < parameters.rsi_low and previous_rsi_values[pair] > parameters.rsi_low

            if not pair_open:
                # Check for opening a position if no position is open
                if rsi_cross_above:
                    position_manager.log(f"Pair {pair} crossed above")
                    alpha_model.set_signal(pair, signal_strength, trailing_stop_loss=parameters.trailing_stop_loss)
            else:
                # We have open position, check for the close condition
                if rsi_cross_below:
                    position_manager.log(f"Pair {pair} crossed below")
                    alpha_model.set_signal(pair, 0)
    
    # Enable trailing stop loss if we have reached the activation level
    if parameters.trailing_stop_loss_activation_level is not None:
       for p in state.portfolio.open_positions.values():
           if p.trailing_stop_loss_pct is None:
               if current_price[p.pair] >= p.get_opening_price() * parameters.trailing_stop_loss_activation_level:
                   p.trailing_stop_loss_pct = parameters.trailing_stop_loss

    # Use alpha model and construct a portfolio of two assets
    alpha_model.select_top_signals(2)
    alpha_model.assign_weights(weight_passthrouh)
    alpha_model.normalise_weights()
    alpha_model.update_old_weights(state.portfolio)
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * parameters.allocation
    alpha_model.calculate_target_positions(position_manager, portfolio_target_value)
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=parameters.minimum_rebalance_trade_percent * portfolio.get_total_equity(),
    )

    #
    # Visualisations
    #

    visualisation = state.visualisation  # Helper class to visualise strategy output

    visualisation.plot_indicator(
        timestamp,
        f"ETH",
        PlotKind.technical_indicator_detached,
        current_price[eth_pair],
        colour="blue",
    )

    # Draw BTC + ETH RSI between its trigger zones for this pair of we got a valid value for RSI for this pair

    # BTC RSI daily
    if current_rsi_values[btc_pair]:
        visualisation.plot_indicator(
            timestamp,
            f"RSI BTC",
            PlotKind.technical_indicator_detached,
            current_rsi_values[btc_pair],
            colour="orange",
        )
            

        # Low (vertical line)
        visualisation.plot_indicator(
            timestamp,
            f"RSI low trigger",
            PlotKind.technical_indicator_overlay_on_detached,
            parameters.rsi_low,
            detached_overlay_name=f"RSI BTC",
            plot_shape=PlotShape.horizontal_vertical,
            colour="red",
            label=PlotLabel.hidden,
        )

        # High (vertical line)
        visualisation.plot_indicator(
            timestamp,
            f"RSI high trigger",
            PlotKind.technical_indicator_overlay_on_detached,
            parameters.rsi_high,
            detached_overlay_name=f"RSI BTC",
            plot_shape=PlotShape.horizontal_vertical,
            colour="red",
            label=PlotLabel.hidden,
        )        
        
    # ETH RSI daily
    if current_rsi_values[eth_pair]:
        visualisation.plot_indicator(
            timestamp,
            f"RSI ETH",
            PlotKind.technical_indicator_overlay_on_detached,
            current_rsi_values[eth_pair],
            colour="blue",
            label=PlotLabel.hidden,
            detached_overlay_name=f"RSI BTC",
        )

    if eth_btc_yesterday is not None:
        visualisation.plot_indicator(
            timestamp,
            f"ETH/BTC",
            PlotKind.technical_indicator_detached,
            eth_btc_yesterday,
            colour="grey",
        )
    
    if eth_btc_rsi_yesterday is not None:
        visualisation.plot_indicator(
            timestamp,
            f"ETH/BTC RSI",
            PlotKind.technical_indicator_detached,
            eth_btc_rsi_yesterday,
            colour="grey",
        )

    # Slow function warning
    # state.visualisation.add_calculations(timestamp, alpha_model.to_dict())  # Record alpha model thinking

    position_manager.log(
        f"BTC RSI: {current_rsi_values[btc_pair]}, BTC RSI yesterday: {previous_rsi_values[btc_pair]}, ETH/BTC: {eth_btc_yesterday}, ETH/BTC RSI: {eth_btc_rsi_yesterday}",
    )

    return trades

# Strategy indicators

- Set up indicators used to decide trades
- Indicator data will be precalculated and cached between strategy runs 
- The actual calculation is run (if needed) later on `run_backtest_inline()` call

In [ ]:
import pandas_ta

from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse


def calculate_eth_btc(strategy_universe: TradingStrategyUniverse):
    """ETH/BTC price series.
    
    - A custom indicator we add
    """
    eth = strategy_universe.get_pair_by_human_description(our_pairs[0])
    btc = strategy_universe.get_pair_by_human_description(our_pairs[1])    
    btc_price = strategy_universe.data_universe.candles.get_candles_by_pair(eth.internal_id)
    eth_price = strategy_universe.data_universe.candles.get_candles_by_pair(btc.internal_id)
    series = eth_price["close"] / btc_price["close"]  # Divide two series
    return series


def calculate_eth_btc_rsi(strategy_universe: TradingStrategyUniverse, length: int):
    """ETH/BTC RSI series.
    
    - A custom indicator we add
    """
    eth = strategy_universe.get_pair_by_human_description(our_pairs[0])
    btc = strategy_universe.get_pair_by_human_description(our_pairs[1])    
    btc_price = strategy_universe.data_universe.candles.get_candles_by_pair(eth.internal_id)
    eth_price = strategy_universe.data_universe.candles.get_candles_by_pair(btc.internal_id)
    series = eth_price["close"] / btc_price["close"]  # Divide two series
    return pandas_ta.rsi(series, length=length)


def create_indicators(parameters: StrategyParameters, indicators: IndicatorSet, strategy_universe: TradingStrategyUniverse, execution_context: ExecutionContext):
    """Set up 3 indicators for this strategy"""    
    indicators.add("rsi", pandas_ta.rsi, {"length": parameters.rsi_bars})  # Calculate RSI for all pairs    
    indicators.add("eth_btc", calculate_eth_btc, source=IndicatorSource.strategy_universe)  # Custom: ETC/BTC close price    
    indicators.add("eth_btc_rsi", calculate_eth_btc_rsi, parameters={"length": parameters.eth_btc_rsi_bars}, source=IndicatorSource.strategy_universe)  # Custom: ETC/BTC RSI


# Parameters

- Set up strategy parameters for this backtest run

In [ ]:
class MyStrategyParameters:
    cycle_duration = CycleDuration.cycle_8h
    rsi_bars = 12
    eth_btc_rsi_bars = 5  # The length of ETH/BTC RSI
    rsi_high = 67
    rsi_low = 60
    allocation = 0.98 # Allocate 90% of cash to each position
    lookback_candles = 120
    minimum_rebalance_trade_percent = 0.05  # Don't do trades that would have less than 500 USD value change
    initial_cash = 10_000 # Start with 10k USD
    trailing_stop_loss = None
    trailing_stop_loss_activation_level = None

# Backtest

Run the backtest.

In [ ]:
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, _, diagnostics_data = run_backtest_inline(
    name="RSI multipair",
    engine_version="0.5",
    decide_trades=decide_trades,
    create_indicators=create_indicators,
    client=client,
    universe=strategy_universe,
    parameters=StrategyParameters.from_class(MyStrategyParameters),
    strategy_logging=False,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

# Diagnostics

- Some quick smoke checks about the execution
- Peak inside the raw data of our custom indicators

In [ ]:
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInputIndicators

indicators: StrategyInputIndicators = diagnostics_data["indicators"]
eth_btc = indicators.get_indicator_series("eth_btc")
display(eth_btc)

In [ ]:
eth_btc_rsi = indicators.get_indicator_series("eth_btc_rsi")
display(eth_btc_rsi)

# Equity curve

In [ ]:
from tradeexecutor.visual.benchmark import visualise_equity_curve_benchmark

btc_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[0])
eth_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[1])

benchmark_indexes = pd.DataFrame({
    "BTC": strategy_universe.data_universe.candles.get_candles_by_pair(btc_pair)["close"],
    "ETH": strategy_universe.data_universe.candles.get_candles_by_pair(eth_pair)["close"],
})
benchmark_indexes["BTC"].attrs = {"colour": "orange", "name": "Buy and hold BTC"}
benchmark_indexes["ETH"].attrs = {"colour": "blue", "name": "Buy and hold ETH"}

fig = visualise_equity_curve_benchmark(
    name=state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    benchmark_indexes=benchmark_indexes,
    height=800,
    log_y=True,
)

fig.show()

# Strategy visualisation

- Show indicators and trades

In [ ]:
from tradeexecutor.visual.single_pair import visualise_single_pair
from tradingstrategy.charting.candle_chart import VolumeBarMode

# Use BTC pair as the primary price source
# Get BTC pair
btc_pair = pairs.get_pair_by_human_description(our_pairs[0])

figure = visualise_single_pair(
    state,
    strategy_universe.data_universe.candles,
    pair_id=btc_pair.pair_id,
    volume_bar_mode=VolumeBarMode.hidden,
    height = 2000,
)

figure.show()

# Benchmark

- Benchmark the strategy against buy and hold Bitcoin
- We are compensating volatility and draw down for lesser returns



In [ ]:
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns, generate_buy_and_hold_returns
from tradeexecutor.analysis.advanced_metrics import visualise_advanced_metrics, AdvancedMetricsMode

equity = calculate_equity_curve(state)
returns = calculate_returns(equity)
benchmark_returns = generate_buy_and_hold_returns(benchmark_indexes["ETH"])

metrics = visualise_advanced_metrics(
    returns,
    mode=AdvancedMetricsMode.full,
    benchmark=benchmark_returns,
)

display(metrics)